# Data Cleaning

Filling in the blanks as best as we can.

## Imports

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("train.csv")

## Finding out where the blanks are

In [3]:
df.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

__Plan of action__:
- Age and Cabin have quite a few null values.
- Age can be filled in with the average across different groups.
- Null Values in Cabin can just be filled in with a "U" for Unknown. Can't really guess this column.
- The two missing values in Embarked do not have to be dealt with as I will be dropping this column anyway. I will however put a "U" in them for completeness.

## Cabin

In [4]:
df["Cabin"] = df["Cabin"].fillna("U").apply(str).apply(lambda x: x[0])
df["Cabin"].isna().sum()

0

## Embarked

In [5]:
df["Embarked"] = df["Embarked"].fillna("C")
df["Embarked"].isna().sum()

0

Before we can fill in the age column, we should first extract the titles from the Name column as they can provide some insight about the age of a person.

## Name

In [6]:
def extract_title(name):
    return name.split(",")[1].split(".")[0].strip()

df["Title"] = df["Name"].apply(extract_title)

In [7]:
df["Title"].unique()

array(['Mr', 'Mrs', 'Miss', 'Master', 'Don', 'Rev', 'Dr', 'Mme', 'Ms',
       'Major', 'Lady', 'Sir', 'Mlle', 'Col', 'Capt', 'the Countess',
       'Jonkheer'], dtype=object)

In [8]:
mean_age = df.groupby(["Title"])["Age"].mean()
mean_age

Title
Capt            70.000000
Col             58.000000
Don             40.000000
Dr              42.000000
Jonkheer        38.000000
Lady            48.000000
Major           48.500000
Master           4.574167
Miss            21.773973
Mlle            24.000000
Mme             24.000000
Mr              32.368090
Mrs             35.898148
Ms              28.000000
Rev             43.166667
Sir             49.000000
the Countess    33.000000
Name: Age, dtype: float64

In [9]:
title_dict = {
    "Mr": "Mr",
    "Mrs": "Mrs",
    "Miss": "Ms",
    "Master": "Master",
    "Don": "Royalty",
    "Rev": "Royalty",
    "Dr": "Royalty",
    "Mme": "Mrs",
    "Ms": "Ms",
    "Major": "Officer",
    "Lady": "Royalty",
    "Sir": "Royalty",
    "Mlle": "Ms",
    "Col": "Officer",
    "Capt": "Officer",
    "the Countess": "Royalty",
    "Jonkheer": "Royalty",
}

df["Title"] = df["Title"].apply(lambda x: title_dict[x])

## Age

In [10]:
df["Age"].describe()

count    714.000000
mean      29.699118
std       14.526497
min        0.420000
25%       20.125000
50%       28.000000
75%       38.000000
max       80.000000
Name: Age, dtype: float64

In [11]:
df["Age"].isna().sum()

177

In [13]:
df.groupby(["Title", "Pclass"])["Age"].agg(["mean", "count"])

mean  count
Title   Pclass                  
Master  1        5.306667      3
        2        2.258889      9
        3        5.350833     24
Mr      1       41.580460     87
        2       32.768293     82
        3       28.724891    229
Mrs     1       40.400000     35
        2       33.682927     41
        3       33.515152     33
Ms      1       29.744681     47
        2       22.560606     33
        3       16.123188     69
Officer 1       56.600000      5
Royalty 1       42.555556      9
        2       42.000000      8

In case of males there is a different title for younger males. However, for all unmarried women we have the title of "Ms". This can contain a large range of ages. As demonstrated below.

In [20]:
print("Unmarried females travelling with parents: ", df.loc[(df["Title"] == "Ms") & (df["Parch"] > 0)]["Age"].mean())
print(f"Unmarried females travelling without parents: ", df.loc[(df["Title"] == "Ms") & (df["Parch"] == 0)]["Age"].mean())

Unmarried females travelling with parents:  11.863636363636363
Unmarried females travelling without parents:  27.68617021276596


In [21]:
df.loc[(df["Title"] == "Ms") & (df["Parch"] > 0), 'Title'] = 'FemaleChild'

In [22]:
df.loc[df["Title"] == "FemaleChild"]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
10,11,1,3,"Sandstrom, Miss. Marguerite Rut",female,4.0,1,1,PP 9549,16.7000,G,S,FemaleChild
24,25,0,3,"Palsson, Miss. Torborg Danira",female,8.0,3,1,349909,21.0750,U,S,FemaleChild
43,44,1,2,"Laroche, Miss. Simonne Marie Anne Andree",female,3.0,1,2,SC/Paris 2123,41.5792,U,C,FemaleChild
58,59,1,2,"West, Miss. Constance Mirium",female,5.0,1,2,C.A. 34651,27.7500,U,S,FemaleChild
68,69,1,3,"Andersson, Miss. Erna Alexandra",female,17.0,4,2,3101281,7.9250,U,S,FemaleChild
...,...,...,...,...,...,...,...,...,...,...,...,...,...
835,836,1,1,"Compton, Miss. Sara Rebecca",female,39.0,1,1,PC 17756,83.1583,E,C,FemaleChild
852,853,0,3,"Boulos, Miss. Nourelain",female,9.0,1,1,2678,15.2458,U,C,FemaleChild
853,854,1,1,"Lines, Miss. Mary Conover",female,16.0,0,1,PC 17592,39.4000,D,S,FemaleChild
863,864,0,3,"Sage, Miss. Dorothy Edith ""Dolly""",female,NaN,8,2,CA. 2343,69.5500,U,S,FemaleChild


In [23]:
grp = df.groupby(["Pclass", "Sex", "Title"], as_index=False)["Age"].mean()
grp

,Pclass,Sex,Title,Age
0,1,female,FemaleChild,21.000000
1,1,female,Mrs,40.400000
2,1,female,Ms,34.258065
3,1,female,Royalty,43.333333
4,1,male,Master,5.306667
5,1,male,Mr,41.580460
6,1,male,Officer,56.600000
7,1,male,Royalty,42.166667
8,2,female,FemaleChild,10.538462
9,2,female,Mrs,33.682927


In [24]:
def fill_age(df):
    return grp[(grp["Pclass"] == df["Pclass"]) & (grp["Sex"] == df["Sex"]) & (grp["Title"] == df["Title"])]["Age"]

In [25]:
df["Age"] = df.apply(lambda x: fill_age(x) if np.isnan(x["Age"]) else x["Age"], axis=1)

In [26]:
df.isna().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
Title          0
dtype: int64

In [27]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,U,S,Mr
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C,C,Mrs
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,U,S,Ms
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C,S,Mrs
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,U,S,Mr


# Test Data

In [28]:
test = pd.read_csv("test.csv")
test.isna().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64